# Objetivo
Serviço que recebe lista de atuais clientes do usuário e retorna novos potenciais clientes. 

# Método

Aqui, para teste. NÃO É A VERSÃO FINAL:

1. Clusterizar banco completo
2. Ler arquivo com lista de IDs de clientes atuais
3. Separar parte para teste
4. Identificar clusters da lista de clientes atuais
5. Buscar novos clientes dos mesmos clusters no banco completo
6. Apresentar resultados
7. Testar se lista de recomendações inclui parte separada para teste

Versão final deve ser:
1. Ler arquivo com lista de IDs de clientes atuais
2. Ler banco clusterizado
3. Identificar clusters da lista de clientes atuais
4. Buscar novos clientes dos mesmos clusters no banco completo
5. Apresentar resultados

# Imports

In [1]:
# imports
import pandas as pd
import numpy as np
import feather

# Carregamento do banco de dados com colunas filtradas

In [68]:
fullmarket = feather.read_dataframe('full_market_reduced_columns.feather')
fullmarket.info()
#Tamanho já mais razoável: 141 MB. Ainda tem que resolver os NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462298 entries, 0 to 462297
Data columns (total 48 columns):
id                                       462298 non-null object
fl_matriz                                462298 non-null bool
de_natureza_juridica                     462298 non-null object
sg_uf                                    462298 non-null object
natureza_juridica_macro                  462298 non-null object
de_ramo                                  462298 non-null object
setor                                    460371 non-null object
idade_empresa_anos                       462298 non-null float64
idade_emp_cat                            462298 non-null object
fl_me                                    462298 non-null bool
fl_sa                                    462298 non-null bool
fl_epp                                   462298 non-null bool
fl_mei                                   462298 non-null bool
fl_ltda                                  462298 non-null bool
dt_s

# Seleção de features e clusterização

In [69]:
feature_list = ['de_nivel_atividade', 'de_ramo', 'nm_meso_regiao', 'setor'] #deixar 'nm_divisao' para depois

In [70]:
# fillna

for feat in feature_list:
    print(feat, '-->>', len(fullmarket[feat].unique().tolist()), ' categorias')
    print(fullmarket[feat].value_counts())
    print('NaN antes: ', sum(fullmarket[feat].isna()))
    fullmarket[feat].fillna('DESCONHECIDO', inplace = True)
    print('NaN depois: ', sum(fullmarket[feat].isna()))
    print('\n')

de_nivel_atividade -->> 5  categorias
MEDIA          217949
ALTA           152245
BAIXA           76080
MUITO BAIXA      4856
Name: de_nivel_atividade, dtype: int64
NaN antes:  11168
NaN depois:  0


de_ramo -->> 33  categorias
COMERCIO VAREJISTA                                   172404
SERVICOS DIVERSOS                                     60318
SERVICOS DE ALOJAMENTO/ALIMENTACAO                    30180
INDUSTRIA DA CONSTRUCAO                               25363
COMERCIO E REPARACAO DE VEICULOS                      22356
SERVICOS ADMINISTRATIVOS                              21326
BENS DE CONSUMO                                       21227
SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS        17371
COMERCIO POR ATACADO                                  16464
TRANSPORTE, ARMAZENAGEM E CORREIO                     15485
SERVICOS DE EDUCACAO                                  11311
SERVICOS DE SAUDE                                      9788
CULTURA, ESPORTE E RECREACAO                        

## Encoder

In [71]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [72]:
# apply le on categorical feature columns
# https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

#portfolio 1 com labelencoder: p1enc
p1enc=pd.DataFrame()
categorical_cols = ['de_nivel_atividade', 'de_ramo', 'nm_meso_regiao', 'setor']
# %*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%
# IMPORTANTE aqui, só três features, para poder visualizar em um gráfico 3D.
# Também porque de_nivel_atividade tem alguns NaN, que tem que ser resolvidos depois
# %*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%
p1enc[categorical_cols] = fullmarket[categorical_cols].apply(lambda col: le.fit_transform(col))

p1enc[categorical_cols].head(5)

,de_nivel_atividade,de_ramo,nm_meso_regiao,setor
0,0,11,7,2
1,1,21,4,5
2,3,32,2,5
3,3,25,2,5
4,0,27,7,5


In [73]:
id_only = fullmarket['id']
del fullmarket

In [74]:
p1enc.shape

(462298, 4)

## Clusterização

Problema com AgglomerativeClustering: não há memória suficiente:
https://stackoverflow.com/questions/51129498/memory-error-during-hierarchical-clustering-python-3-6

https://datascience.stackexchange.com/questions/47889/how-to-run-agglomerativeclustering-on-a-big-data-in-python

Sugestão é amostrar o banco e usar KNN.

Ou usar outro algoritmo, como MiniBatchKMeans ou Birch.
https://github.com/scikit-learn/scikit-learn/issues/5256

https://chrisalbon.com/machine_learning/clustering/minibatch_k-means_clustering/

https://towardsdatascience.com/machine-learning-birch-clustering-algorithm-clearly-explained-fb9838cbeed9

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.Birch.html

Birch clustering algorithm is a memory-efficient, online-learning algorithm provided as an alternative to MiniBatchKMeans.


In [23]:
#Então, vamos amostrar o banco
sample_market=p1enc.sample(n=10000, random_state=17)

from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=50, affinity='euclidean', linkage='ward')
sample_market['cluster'] = cluster.fit_predict(sample_market)
# Com 10000, ok

In [26]:
sample_market.shape

(10000, 5)

In [24]:
sample_market.head(5)

,de_nivel_atividade,de_ramo,nm_meso_regiao,setor,cluster
342914,0,25,12,5,32
198208,0,20,1,5,30
47648,0,5,7,1,41
173551,3,20,16,5,6
131912,3,19,11,5,26


In [25]:
# aqui, estou somando um valor aleatório para fins de visualização.
# Se não for feito, um grupo grande de pontos fica condensado em um único ponto no gráfico
# e perde-se a noção de cluster
tam = len(sample_market)
sample_market['de_nivel_atividade'] = sample_market['de_nivel_atividade'] + \
    ((np.random.rand(tam)-0.5) * 0.5)
sample_market['de_ramo'] = sample_market['de_ramo'] + \
    ((np.random.rand(tam)-0.5) * 0.5)
sample_market['nm_meso_regiao'] = sample_market['nm_meso_regiao'] + \
    ((np.random.rand(tam)-0.5) * 0.5)
##sample_market['cluster'] = sample_market['cluster']*3
sample_market.head(5)

,de_nivel_atividade,de_ramo,nm_meso_regiao,setor,cluster
342914,0.061129,25.246016,12.045782,5,32
198208,0.215616,19.945061,1.248050,5,30
47648,-0.158864,5.226785,7.188015,1,41
173551,3.113360,19.941011,16.164287,5,6
131912,3.113199,19.112421,11.233552,5,26


In [27]:
import plotly.express as px
fig = px.scatter_3d(sample_market, x='de_nivel_atividade',
                    y='de_ramo', z='nm_meso_regiao', color='cluster')
fig.show()

Precisamos do banco completo clusterizado. Como o AgglomerativeClustering não funciona por limitação de memória, vamoa adotar a estratégia de clusterizar uma parte e depois usar KNN para predizer as classes do resto do banco.

In [75]:
#https://stackoverflow.com/questions/51129498/memory-error-during-hierarchical-clustering-python-3-6
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(p1enc, pd.Series(np.zeros(len(p1enc))), 
                     train_size=0.02, random_state=17) 
from sklearn.cluster import AgglomerativeClustering
AC = AgglomerativeClustering(n_clusters=25, linkage='ward')
AC.fit(X_train)
labels = AC.labels_

In [76]:
from sklearn.neighbors import KNeighborsClassifier
KN = KNeighborsClassifier(n_neighbors=3)
KN.fit(X_train,labels)
clusters_fulldata = KN.predict(p1enc)
p1enc['cluster'] = clusters_fulldata

In [77]:
p1enc.shape

(462298, 5)

In [78]:
p1enc['cluster'].value_counts()
#Aqui tem o banco completo, com número reduzido de features, clusterizado

8     35665
4     34257
2     30482
7     29133
16    27061
9     26346
1     26034
6     25737
22    24501
5     24040
17    21810
12    21024
0     19385
10    18218
23    16499
20    14210
24    11716
11    10974
3      8519
14     7419
15     6849
18     6836
13     6647
19     6622
21     2314
Name: cluster, dtype: int64

In [79]:
#Depois do KNN, posso incluir o id no dataframe
fullbase_enc = p1enc
fullbase_enc['id'] = id_only

# Carregamento lista de clientes atuais
Formato: arquivo csv, com IDs dos clientes

In [97]:
current_customers = pd.read_csv('estaticos_portfolio1.csv')
if not 'id' in current_customers.columns:
    print('Error. Check yout file. It should contain a column named "id"') 
else:
    print('Ok. Loaded file. Shape:', current_customers.shape)
    

Ok. Loaded file. Shape: (555, 182)


# Análise dos clientes atuais
Resumo da situação atual e identificação dos clusters

In [98]:
curr_customers_in_base = fullbase_enc[fullbase_enc['id'].isin(current_customers['id'].tolist())]
curr_customers_in_base.shape

(555, 6)

In [99]:
curr_customers_in_base['cluster'].value_counts()
#Essa é a parte interessante.

8     81
4     75
2     71
22    69
7     67
16    59
17    44
10    40
23    32
19    17
Name: cluster, dtype: int64

In [100]:
curr_customers_in_base[curr_customers_in_base['cluster']==8]['nm_meso_regiao'].value_counts()

2    32
4    26
3    14
0     5
1     4
Name: nm_meso_regiao, dtype: int64

In [101]:
sum(p1enc['cluster']==10)

18218

Como fazer aquela avaliação de aderência?

# Busca de clientes novos na base

# Apresentação dos resultados
Ranking e mapa